In [1]:
import pandas as pd
from io import StringIO
import os
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Union, Set, Any
from enum import Enum
import pickle
import pandas as pd
import statsmodels.api as sm
import numpy as np

from nst_scraper import nst_on_ice_scraper, nst_team_on_ice_scraper
from db_utils.nhl_db_utils import update_player_db, check_last_update, append_player_ids, get_player_full_name
from team_utils import get_most_recent_game_id, get_fullname_by_tricode
from game_utils import get_game_boxscore, display_boxscore
from pbp_utils import get_matchup_games

from dotenv import load_dotenv

pd.set_option('display.max_columns', None)
np.set_printoptions(legacy='1.25')

In [2]:
player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025,startdate='', enddate='2024-11-19', rate='y')
player_stats_df.head()

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Suter,STL,D,20,388.52,19.43,0.15,0.46,0.15,0.31,0.62,20.00,2.63,5.88,0.12,7.72,4.32,1.70,0.31,0.46,0.77,1.85,0.93,0.93,0.00,0.0,0.15,3.40,0.62,1.70,2.16,3.40,0.00,0.00,-
1,Brent Burns,CAR,D,17,277.03,16.30,0.00,1.08,0.43,0.65,1.08,33.33,5.63,0.00,0.24,14.94,9.10,4.12,0.43,0.22,1.08,0.87,0.43,0.43,0.00,0.0,0.22,5.20,1.52,0.43,1.73,3.03,0.00,0.00,-
2,Corey Perry,EDM,R,20,204.60,10.23,0.88,0.00,0.00,0.00,0.88,42.86,5.28,16.67,0.65,11.14,7.62,6.45,3.23,0.00,0.29,4.11,1.17,0.59,0.59,0.0,2.05,1.76,0.88,1.47,2.93,1.47,0.00,0.59,0.00
3,Alex Ovechkin,WSH,L,18,215.25,11.96,2.51,1.95,1.11,0.84,4.46,72.73,9.48,26.47,0.91,19.23,13.66,11.15,4.46,0.00,1.39,1.67,0.84,0.84,0.00,0.0,0.00,1.95,1.11,11.43,2.23,0.84,0.00,0.00,-
4,Evgeni Malkin,PIT,C,21,316.17,15.06,0.57,1.71,1.14,0.57,2.28,75.00,5.69,10.00,0.88,11.58,7.59,6.83,3.80,0.00,1.52,1.52,0.76,0.76,0.00,0.0,0.19,3.80,0.57,1.71,3.42,2.85,11.77,15.18,8.29


In [3]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', pos='G', rate='y', stdoi='g')
goalie_stats_df.head()

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,10,471.12,47.11,27.76,25.34,0.913,2.42,0.05,1.81,4.97,3.95,0.795,1.02,-0.11,5.73,5.09,0.889,0.64,0.03,15.66,15.03,0.959,0.64,-0.12,2.04,3.31,38.89,30.84
1,Jonathan Quick,NYR,10,455.50,45.55,28.72,26.61,0.927,2.11,0.45,2.33,7.38,6.59,0.893,0.79,0.56,7.51,6.45,0.860,1.05,-0.18,12.78,12.51,0.979,0.26,0.16,2.24,4.48,35.82,22.75
2,James Reimer,ANA,2,103.60,51.80,30.69,27.80,0.906,2.90,-0.16,3.01,4.05,2.32,0.571,1.74,-1.00,13.32,12.16,0.913,1.16,0.40,13.32,13.32,1.000,0.00,0.44,1.74,4.63,30.81,17.20
3,James Reimer,BUF,4,153.35,38.34,25.04,21.91,0.875,3.13,-0.90,2.33,3.91,1.96,0.500,1.96,-1.24,10.56,9.78,0.926,0.78,0.45,10.17,9.78,0.962,0.39,-0.06,2.74,6.65,34.08,16.25
4,Semyon Varlamov,NYI,10,499.45,49.94,25.35,23.07,0.910,2.28,-0.02,2.36,7.33,6.37,0.869,0.96,0.38,6.01,5.29,0.880,0.72,-0.02,9.85,9.25,0.939,0.60,-0.28,1.92,4.44,35.72,20.05


In [4]:
class Position(Enum):
    C = 'C'
    L = 'L'
    R = 'R'
    D = 'D'
    G = 'G'
    
    @property
    def category(self) -> str:
        if self in {Position.C, Position.L, Position.R}:
            return 'F'
        elif self == Position.D:
            return 'D'
        elif self == Position.G:
            return 'G'
    
    def __str__(self) -> str:
        return self.value

In [5]:
@dataclass
class Player:
    name: str
    team: str
    position: Position
    player_id: Optional[int] = None

    def __str__(self) -> str:
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self) -> Dict[str, Optional[str]]:
        """
        Converts the Player instance into a dictionary.
        
        Returns:
            Dict[str, Optional[str]]: A dictionary representation of the player.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position.value
        }

In [6]:
@dataclass
class Lineup:
    name: str
    forwards: List[Optional[Player]] = field(default_factory=lambda: [None] * 12)
    defense: List[Optional[Player]] = field(default_factory=lambda: [None] * 6)
    goalies: List[Optional[Player]] = field(default_factory=lambda: [None] * 2)
    
    ALLOWED_FORWARD_CATEGORIES = {'F'}
    ALLOWED_DEFENSE_CATEGORY = 'D'
    ALLOWED_GOALIE_CATEGORY = 'G'
    
    def __post_init__(self):
        self.validate_lineup_size()
    
    def validate_lineup_size(self):
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if total_players > 20:
            raise ValueError(f"Total number of players ({total_players}) exceeds the hard limit of 20.")
    
    def add_player(
        self,
        category: str,
        player: Player,
        slot: int,
        allowed_categories: Union[str, Set[str]],
        max_slots: int
    ):
        """
        Adds a player to the specified category and slot after validating their position category.
        Ensures that the total number of players does not exceed 20.
        """
        if isinstance(allowed_categories, str):
            allowed_categories = {allowed_categories}
        elif isinstance(allowed_categories, set):
            allowed_categories = allowed_categories
        else:
            raise TypeError("allowed_categories must be a string or a set of strings.")
        
        if player.position.category not in allowed_categories:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position.value}' "
                f"to {category}. Allowed categories: {', '.join(allowed_categories)}."
            )
        
        if not 0 <= slot < max_slots:
            raise IndexError(f"{category.capitalize()} slot must be between 0 and {max_slots - 1}.")
        
        current_category = getattr(self, category)
        if current_category[slot]:
            existing_player = current_category[slot].name
            print(f"Warning: Slot {slot + 1} in {category} is already occupied by '{existing_player}'. Overwriting.")
        
        # Check total players before adding
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if current_category[slot] is None and total_players >= 20:
            raise ValueError("Cannot add more players. The lineup has reached the hard limit of 20 players.")
        
        current_category[slot] = player
        setattr(self, category, current_category)
        print(f"Added player '{player.name}' to {category.capitalize()} slot {slot + 1}.")
    
    def add_forward(self, player: Player, slot: int):
        self.add_player(
            category='forwards',
            player=player,
            slot=slot,
            allowed_categories=self.ALLOWED_FORWARD_CATEGORIES,
            max_slots=len(self.forwards)
        )
    
    def add_defense(self, player: Player, slot: int):
        self.add_player(
            category='defense',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_DEFENSE_CATEGORY},
            max_slots=len(self.defense)
        )
    
    def set_goalie(self, player: Player, slot: int):
        self.add_player(
            category='goalies',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_GOALIE_CATEGORY},
            max_slots=len(self.goalies)
        )
    
    def adjust_slots(self, category: str, delta: int):
        """
        Adjusts the number of slots in the specified category by delta.
        Allows ±1 adjustment only.
        
        Args:
            category (str): The category to adjust ('forwards' or 'defense').
            delta (int): The change in number of slots (+1 or -1).
        """
        if category not in {'forwards', 'defense'}:
            raise ValueError("Can only adjust 'forwards' or 'defense' categories.")
        if delta not in {-1, 1}:
            raise ValueError("Delta must be either +1 or -1.")
        
        current_slots = getattr(self, category)
        new_slot_count = len(current_slots) + delta
        
        if category == 'forwards':
            if not (11 <= new_slot_count <= 13):
                raise ValueError("Number of forwards can only vary by ±1 from the default of 12.")
        elif category == 'defense':
            if not (5 <= new_slot_count <= 7):
                raise ValueError("Number of defensemen can only vary by ±1 from the default of 6.")
        
        if delta == 1:
            current_slots.append(None)
        elif delta == -1:
            removed_player = current_slots.pop()
            if removed_player:
                print(f"Removed player '{removed_player.name}' from {category}.")
        
        setattr(self, category, current_slots)
        print(f"Adjusted {category} slots to {len(getattr(self, category))}.")
        self.validate_lineup_size()
    
    def display_lineup(self):
        """
        Prints the current lineup in a structured format.
        """
        print(f"Lineup: {self.name}\n")
        
        for category, title in [('forwards', 'Forwards'), ('defense', 'Defense'), ('goalies', 'Goalies')]:
            print(f"{title}:")
            for idx, player in enumerate(getattr(self, category), start=1):
                player_info = str(player) if player else 'Empty'
                print(f"  Slot {idx}: {player_info}")
            print()
    
    def to_dataframe(self) -> pd.DataFrame:
        """
        Converts the lineup into a pandas DataFrame.
        Conditionally includes player attributes if they are present.
        """
        data = []
        for category, pos in [('forwards', 'f'), ('defense', 'd'), ('goalies', 'g')]:
            for idx, player in enumerate(getattr(self, category), start=1):
                player_dict = {
                    'Position': f"{pos}{idx}",
                    'Player': player.name if player else 'Empty'
                }
                # Conditionally add 'player_id' if it exists
                if player and player.player_id is not None:
                    player_dict['Player ID'] = player.player_id
                data.append(player_dict)
        
        df = pd.DataFrame(data)
        
        # Optionally, remove columns where all values are NaN
        df.dropna(axis=1, how='all', inplace=True)
        
        return df
    
    def to_transposed_dataframe(self) -> pd.DataFrame:
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        and the row contains the corresponding player names.
        Conditionally includes additional player attributes if they are present.
        """
        df = self.to_dataframe()
        
        # Initialize dictionaries to hold player names and optional IDs
        player_data = {}
        id_data = {}
        
        for _, row in df.iterrows():
            pos = row['Position']
            player_name = row['Player']
            player_data[pos] = player_name
            
            # Handle 'Player ID' if it exists
            if 'Player ID' in row and pd.notna(row['Player ID']):
                id_data[f"{pos}_ID"] = row['Player ID']
        
        # Combine player names and IDs into a single dictionary
        transposed_data = {**player_data, **id_data}
        
        # Create the transposed DataFrame with a single row
        transposed_df = pd.DataFrame([transposed_data])
        
        return transposed_df

In [7]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    try:
        # !! Take only first position if multiple are listed, this is a hack to deal with the fact that some players have multiple positions listed
        position = row['position'].split(',')[0].strip()
        position_enum = Position(position)  # Convert abbreviation to Position Enum
    except ValueError:
        print(f"Invalid position '{row['position']}' for player '{row['player']}'. Skipping.")
        continue
    
    player = Player(
        name=row['player'],
        team=row['team'],
        position=position_enum
        # player_id is not set initially
    )
    player_list.append(player)

# Convert player list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in player_list]).head()

,name,team,position
0,Ryan Suter,STL,D
1,Brent Burns,CAR,D
2,Corey Perry,EDM,R
3,Alex Ovechkin,WSH,L
4,Evgeni Malkin,PIT,C


In [8]:
goalie_stats_df.head()

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,10,471.12,47.11,27.76,25.34,0.913,2.42,0.05,1.81,4.97,3.95,0.795,1.02,-0.11,5.73,5.09,0.889,0.64,0.03,15.66,15.03,0.959,0.64,-0.12,2.04,3.31,38.89,30.84
1,Jonathan Quick,NYR,10,455.50,45.55,28.72,26.61,0.927,2.11,0.45,2.33,7.38,6.59,0.893,0.79,0.56,7.51,6.45,0.860,1.05,-0.18,12.78,12.51,0.979,0.26,0.16,2.24,4.48,35.82,22.75
2,James Reimer,ANA,2,103.60,51.80,30.69,27.80,0.906,2.90,-0.16,3.01,4.05,2.32,0.571,1.74,-1.00,13.32,12.16,0.913,1.16,0.40,13.32,13.32,1.000,0.00,0.44,1.74,4.63,30.81,17.20
3,James Reimer,BUF,4,153.35,38.34,25.04,21.91,0.875,3.13,-0.90,2.33,3.91,1.96,0.500,1.96,-1.24,10.56,9.78,0.926,0.78,0.45,10.17,9.78,0.962,0.39,-0.06,2.74,6.65,34.08,16.25
4,Semyon Varlamov,NYI,10,499.45,49.94,25.35,23.07,0.910,2.28,-0.02,2.36,7.33,6.37,0.869,0.96,0.38,6.01,5.29,0.880,0.72,-0.02,9.85,9.25,0.939,0.60,-0.28,1.92,4.44,35.72,20.05


In [9]:
# Creating Player instances from the goalie_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=Position.G  # Set position to 'G' for goalies
        # player_id is not set initially
    )
    goalie_list.append(player)

# Convert goalie list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,ANA,G
3,James Reimer,BUF,G
4,Semyon Varlamov,NYI,G


In [10]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-12-19'

In [11]:
# Load environment variables from .env file
load_dotenv()

db_prefix = 'NHL_DB_'

# Construct the database configuration dictionary
# db_prefix = {
#     'dbname': os.getenv(f'{db_prefix}NAME'),
#     'user': os.getenv(f'{db_prefix}USER'),
#     'password': os.getenv(f'{db_prefix}PASSWORD'),
#     'host': os.getenv(f'{db_prefix}HOST'),
#     'port': os.getenv(f'{db_prefix}PORT')
# }

In [12]:
# Check the last update time of the players database
last_update = check_last_update(db_prefix)

INFO:db_utils.base_utils:Database connection established.
INFO:db_utils.nhl_db_utils:Last database update was on: 2024-12-19
INFO:db_utils.base_utils:Database connection closed.


In [13]:
# Convert last_update to datetime
last_update_dt = datetime.strptime(last_update, '%Y-%m-%d')
today_dt = datetime.strptime(today_datetime.strftime('%Y-%m-%d'), '%Y-%m-%d')
yesterday_dt = datetime.strptime(yesterday, '%Y-%m-%d')

# Only update if last update was before yesterday
if last_update_dt not in [today_dt, yesterday_dt]:
    # Update the player database from last update to yesterday
    update_player_db(last_update, yesterday, db_prefix, skip_existing=True)
else:
    print(f"No need to update the player database. Last update was on: {last_update}")


No need to update the player database. Last update was on: 2024-12-19


In [14]:
def add_player_to_lineup(lineup: Lineup, player: Player, category: str):
    """
    Adds a player to the lineup in the specified category, handling potential errors.
    
    Args:
        lineup (Lineup): The lineup object.
        player (Player): The player to add.
        category (str): The category ('forwards', 'defense', 'goalies').
    """
    try:
        if category == 'forwards':
            slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
            lineup.add_forward(player, slot)
        elif category == 'defense':
            slot = next(i for i, p in enumerate(lineup.defense) if p is None)
            lineup.add_defense(player, slot)
        elif category == 'goalies':
            slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, slot)
        else:
            print(f"Unknown category '{category}'.")
    except StopIteration:
        print(f"No available slots to add player '{player.name}' in category '{category}'.")
    except ValueError as ve:
        print(ve)
    except IndexError as ie:
        print(ie)

In [15]:
# TODO this function just creates a lineup from the player_list and goalie_list
def create_lineup(team) -> Lineup:
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

# my_lineup = create_lineup('TOR')

In [16]:
def create_flexible_lineup(team: str) -> Lineup:
    """
    Creates a flexible lineup for the specified team, allowing for ±1 forward or defense spot.
    
    Args:
        team (str): The team name to filter players.
    
    Returns:
        Lineup: The configured lineup object.
    """
    lineup = Lineup(f"Flexible Lineup for {team}")
    
    # Example logic to adjust slots based on team strategy
    # Here, we simply allow flexibility; implement specific rules as needed
    allow_extra_forward = True  # Example condition
    allow_extra_defense = False  # Example condition
    
    if allow_extra_forward:
        lineup.adjust_slots('forwards', 1)  # Increase forwards to 13
    elif allow_extra_defense:
        lineup.adjust_slots('defense', 1)  # Increase defense to 7
    
    # Add players to the lineup
    forward_count = 0
    defense_count = 0
    goalie_count = 0
    
    for player in player_list:
        if player.team == team:
            if player.position.category == 'F' and forward_count < len(lineup.forwards):
                try:
                    lineup.add_forward(player, forward_count)
                    forward_count += 1
                except ValueError:
                    continue
            elif player.position.category == 'D' and defense_count < len(lineup.defense):
                try:
                    lineup.add_defense(player, defense_count)
                    defense_count += 1
                except ValueError:
                    continue
    
    for goalie in goalie_list:
        if goalie.team == team and goalie_count < len(lineup.goalies):
            try:
                lineup.set_goalie(goalie, goalie_count)
                goalie_count += 1
            except ValueError:
                continue
    
    return lineup

# Create and display a flexible lineup for 'TOR'
# flexible_lineup = create_flexible_lineup('TOR')
# flexible_lineup.display_lineup()

In [17]:
def assign_player_ids_to_lineup(transposed_lineup_df, db_prefix):
    """
    Processes the transposed_lineup_df DataFrame to append player IDs next to each player's name.
    
    This function performs the following steps:
        1. Extracts all unique player names from the lineup DataFrame.
        2. Creates Player instances for each player name.
        3. Uses the append_player_ids function to assign player IDs to each Player object.
        4. Inserts new columns into the DataFrame with the corresponding player IDs.
    
    Args:
        transposed_lineup_df (pd.DataFrame): The transposed lineup DataFrame with player names.
        db_prefix (dict): prefix of the database name to use for player_id lookup
    
    Returns:
        pd.DataFrame: The updated DataFrame with player IDs added.
    """
    # Ensure the DataFrame has only one row
    if transposed_lineup_df.shape[0] != 1:
        raise ValueError("transposed_lineup_df should contain exactly one row representing the lineup.")

    # Extract player names from the DataFrame
    player_columns = transposed_lineup_df.columns
    player_names = transposed_lineup_df.iloc[0].tolist()

    # Create Player instances
    player_list = []
    for name in player_names:
        if name != 'Empty':
            player = Player(name=name, team=None, position=None)  # Team and position can be set if available
            player_list.append(player)

    # Append player IDs using the existing function
    append_player_ids(player_list, db_prefix)

    # Create a mapping from player name to player_id
    name_to_id = {player.name: player.player_id for player in player_list if player.player_id is not None}

    # Initialize a copy of the DataFrame to avoid modifying the original
    updated_df = transposed_lineup_df.copy()

    # Insert player_id columns adjacent to each player name column
    for col in player_columns:
        player_name = updated_df.at[0, col]
        if player_name != 'Empty':
            player_id = name_to_id.get(player_name, None)
            id_column = f"{col}_id"
            updated_df[id_column] = player_id
        else:
            id_column = f"{col}_id"
            updated_df[id_column] = None  # Assign None for empty slots

    return updated_df

In [18]:
def get_skater_stats(lineup: Lineup, player_stats_df: pd.DataFrame, filter: Optional[str] = None) -> pd.DataFrame:
    """
    Gets stats for players in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing players
        player_stats_df (pd.DataFrame): DataFrame with player statistics
        filter (str, optional): Type of filter to apply on the stats.
        Defaults to None, which keeps all stats. If 'shots', only shot-related statistics are kept.
        
    Returns:
        pd.DataFrame: Player statistics ordered according to lineup positions
    """
    # Apply filter if specified
    if filter == 'shots':
        # Define shot-related columns to keep
        shot_columns = ['player', 'team', 'position', 'gp', 'toi', 'toi/gp','shots/60', 'icf/60', 'iff/60'] 
        player_stats_df = player_stats_df[shot_columns]
    
    # Create ordered list of players (forwards then defense)
    players = []
    # Add forwards in order
    players.extend([p for p in lineup.forwards if p])
    # Add defense in order 
    players.extend([p for p in lineup.defense if p])
    
    # Create ordered list of player names
    player_names = [player.name for player in players]
    
    # Filter stats and reorder to match lineup order
    stats_df = player_stats_df[player_stats_df['player'].isin(player_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(player_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

# lineup_player_stats = get_skater_stats(my_lineup, player_stats_df)

In [19]:
def get_goalie_stats(lineup: Lineup, goalie_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Gets stats for goalies in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing goalies
        goalie_stats_df (pd.DataFrame): DataFrame with goalie statistics
        
    Returns:
        pd.DataFrame: Goalie statistics ordered according to lineup positions
    """
    # Create ordered list of goalies
    goalies = [goalie for goalie in lineup.goalies if goalie]
    goalie_names = [goalie.name for goalie in goalies]
    
    # Filter stats and reorder to match lineup order
    stats_df = goalie_stats_df[goalie_stats_df['player'].isin(goalie_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(goalie_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

In [20]:
def extract_team_lineup(team: str, reference_date: Optional[str] = None) -> Lineup:
    """
    Extracts the most recent lineup for the specified team based on the latest game data.

    This function performs the following steps:
        1. Determines the reference date (defaults to yesterday if not provided).
        2. Retrieves the most recent game ID for the team using `get_most_recent_game_id`.
        3. Fetches the game boxscore data using `get_game_boxscore`.
        4. Processes the boxscore to obtain skaters and goalies using `display_boxscore`.
        5. Constructs and returns a `Lineup` object populated with the team's players.

    Args:
        team (str): The three-letter team code (e.g., 'TOR').
        reference_date (Optional[str]): The reference date in 'YYYY-MM-DD' format. Defaults to yesterday's date.

    Returns:
        Lineup: A `Lineup` object containing the team's players from the most recent game.

    Raises:
        ValueError: If no recent game is found for the team or if the team is not part of the retrieved game.
    """
    # Step 1: Determine the reference date
    if reference_date is None:
        today_datetime = datetime.now()
        yesterday_datetime = today_datetime - timedelta(days=1, hours=6)  # Adjust for UTC offset if necessary
        reference_date = yesterday_datetime.strftime('%Y-%m-%d')

    # Step 2: Retrieve the most recent game ID for the team
    game_id = get_most_recent_game_id(team, reference_date)
    if game_id is None:
        raise ValueError(f"No recent game found for team '{team}' before {reference_date}.")

    # Print the game_id
    print(f"Game ID: {game_id}")

    # Step 3: Fetch the game boxscore data
    game_data = get_game_boxscore(game_id, clean=False)

    # Step 4: Process the boxscore to obtain skaters and goalies
    away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(game_data)

    # Extract team abbrevs to determine if the team is home or away
    away_team_code = game_data.get('awayTeam', {}).get('abbrev')
    home_team_code = game_data.get('homeTeam', {}).get('abbrev')

    if not away_team_code or not home_team_code:
        raise ValueError("Team abbreviations not found in game data.")

    if team.upper() == away_team_code.upper():
        team_side = 'Away'
        skaters = away_skaters
        goalies = away_goalies
    elif team.upper() == home_team_code.upper():
        team_side = 'Home'
        skaters = home_skaters
        goalies = home_goalies
    else:
        raise ValueError(f"Team '{team}' not found in game ID {game_id}.")

    # Step 5: Construct the Lineup object
    lineup = Lineup(name=f"{team.upper()} Lineup from Game {game_id}")

    # Add Skaters to the Lineup
    for _, skater in skaters.iterrows():
        try:
            position_enum = Position(skater['position'])  # Convert to Position Enum
        except ValueError:
            print(f"Invalid position '{skater['position']}' for player '{skater['name']}'. Skipping.")
            continue

        player = Player(
            player_id=skater['playerId'],
            name=get_player_full_name(skater['playerId'], db_prefix, suppress_log=True),
            team=team.upper(),
            position=position_enum
        )

        # Add player to the appropriate category in the lineup
        if player.position.category == 'F':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
                lineup.add_forward(player, empty_slot)
            except StopIteration:
                print(f"No available forward slot to add player '{player.name}'.")
        elif player.position.category == 'D':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.defense) if p is None)
                lineup.add_defense(player, empty_slot)
            except StopIteration:
                print(f"No available defense slot to add player '{player.name}'.")
        else:
            print(f"Player '{player.name}' has an unrecognized category '{player.position.category}'. Skipping.")

    # Add Goalies to the Lineup
    for _, goalie in goalies.iterrows():
        player = Player(
            player_id=goalie['playerId'],
            name=get_player_full_name(goalie['playerId'], db_prefix, suppress_log=True),
            team=team.upper(),
            position=Position.G
        )
        try:
            empty_slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, empty_slot)
        except StopIteration:
            print(f"No available goalie slot to add player '{player.name}'.")

    return lineup
# chicago_lineup = extract_team_lineup('CHI', '2024-11-22')

In [21]:
def calculate_min_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the average time on ice per game as a percentage of total game time (60 minutes).
    
    Args:
        df (pd.DataFrame): DataFrame containing 'toi' and 'gp' columns
        
    Returns:
        pd.DataFrame: Original DataFrame with new 'min%' column added
    """
    df_copy = df.copy()
    df_copy['min%'] = (df_copy['toi'] / df_copy['gp'] / 300 * 100).round(2)
    return df_copy

# # Apply the function to lineup_player_stats
# lineup_player_stats = calculate_min_percentage(lineup_player_stats)
# lineup_player_stats

In [22]:
def sum_min_percentage(df: pd.DataFrame) -> float:
    """
    Calculates the sum of the 'min%' column in the given DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame containing the 'min%' column.
        
    Returns:
        float: The total sum of the 'min%' values.
        
    Raises:
        KeyError: If the 'min%' column is not present in the DataFrame.
    """
    if 'min%' not in df.columns:
        raise KeyError("The DataFrame does not contain a 'min%' column.")
    
    total_min_percentage = df['min%'].sum()
    return total_min_percentage

# Example usage:
# total_min_percentage = sum_min_percentage(lineup_player_stats)
# print(f"Total min%: {total_min_percentage}")

In [23]:
def calculate_adj_min(df: pd.DataFrame, total_min_percentage: float) -> None:
    """
    Calculates the adjusted minimum (adj_min) for each player based on their min% and the total min%.
    
    The formula used is:
        adj_min = (min% / total_min_percentage) * 300
    
    Args:
        df (pd.DataFrame): DataFrame containing the 'min%' column.
        total_min_percentage (float): The total sum of the 'min%' column.
        
    Raises:
        KeyError: If the 'min%' column is not present in the DataFrame.
        ValueError: If total_min_percentage is not a positive number.
    """
    if 'min%' not in df.columns:
        raise KeyError("The DataFrame does not contain a 'min%' column.")
    
    if total_min_percentage <= 0:
        raise ValueError("total_min_percentage must be a positive number.")
    
    # Calculate and append the 'adj_min' column
    df['adj_min'] = ((df['min%'] / total_min_percentage) * 300).round(2)

# # Example usage:
# calculate_adj_min(lineup_player_stats, total_min_percentage)
# print(lineup_player_stats[['player', 'min%', 'adj_min']])

In [24]:
# Define file paths to load the model and transformer
model_filepath = 'models/polynomial_model_degree_1.pkl'
poly_filepath = 'models/polynomial_features_degree_1.pkl'

# Load the regression model
with open(model_filepath, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
print(f"Model loaded from {model_filepath}")

# Load the PolynomialFeatures transformer
with open(poly_filepath, 'rb') as poly_file:
    loaded_poly = pickle.load(poly_file)
print(f"PolynomialFeatures transformer loaded from {poly_filepath}")

# Function to make predictions using the loaded model and transformer
def predict_gpm(new_ixg60_value, model, poly, x_col='ixg60'):
    """
    Predicts 'gpm' using the loaded model and polynomial transformer.
    
    Parameters:
        new_ixg60_value (float): The new ixg60 value for prediction.
        model (RegressionResults): The loaded regression model.
        poly (PolynomialFeatures): The loaded polynomial features transformer.
        x_col (str): The name of the independent variable column. Defaults to 'ixg60'.
        
    Returns:
        predicted_gpm (float): The predicted gpm value.
    """
    # Prepare the input data
    X_new = np.array([[new_ixg60_value]])
    X_new_poly = poly.transform(X_new)
    X_new_poly_const = sm.add_constant(X_new_poly, has_constant='add')
    
    # Create DataFrame with appropriate column names
    feature_names = ['const'] + poly.get_feature_names_out([x_col]).tolist()
    new_data = pd.DataFrame(X_new_poly_const, columns=feature_names)
    
    # Predict
    predicted_gpm = model.predict(new_data)
    return predicted_gpm.iloc[0]

# Example: Predicting 'gpm' for a new ixg60 value
# new_ixg60_value = 50
# predicted_gpm = predict_gpm(new_ixg60_value, loaded_model, loaded_poly)
# print(f"Predicted GPM for ixg60={new_ixg60_value}: {predicted_gpm:.4f}")

Model loaded from models/polynomial_model_degree_1.pkl
PolynomialFeatures transformer loaded from models/polynomial_features_degree_1.pkl


In [25]:
# Function to predict GPM for each player and add it to the DataFrame
def add_gpm_to_lineup(lineup_df, model, poly):
    """
    Adds a 'gpm' column to the lineup_player_stats DataFrame using the predict_gpm function.
    
    Args:
        lineup_df (pd.DataFrame): DataFrame containing 'ixg/60' column.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        
    Returns:
        pd.DataFrame: Updated DataFrame with 'gpm' column added.
    """
    # Define a helper function to handle potential missing or invalid values
    def safe_predict(ixg_60):
        if pd.isna(ixg_60):
            return np.nan
        try:
            return predict_gpm(ixg_60, model, poly)
        except Exception as e:
            print(f"Error predicting GPM for ixg_60={ixg_60}: {e}")
            return np.nan
    
    # Apply the predict_gpm function to each 'ixg/60' value
    lineup_df['gpm'] = lineup_df['ixg/60'].apply(safe_predict)
    return lineup_df

# # Apply the function to add 'gpm' to your DataFrame
# lineup_player_stats = add_gpm_to_lineup(lineup_player_stats, loaded_model, loaded_poly)

# # Display the updated DataFrame with 'gpm'
# print(lineup_player_stats[['player', 'ixg/60', 'gpm']])

In [26]:
def calculate_x_goals(lineup_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates 'x_goals' by multiplying 'gpm' with 'adj_min' for each player.
    
    Args:
        lineup_stats_df (pd.DataFrame): DataFrame containing 'gpm' and 'adj_min' columns.
        
    Returns:
        pd.DataFrame: DataFrame with the new 'x_goals' column added.
        
    Raises:
        KeyError: If 'gpm' or 'adj_min' columns are not present in the DataFrame.
        TypeError: If 'gpm' or 'adj_min' contain non-numeric data.
    """
    # Check if required columns exist
    required_columns = {'gpm', 'adj_min'}
    missing_columns = required_columns - set(lineup_stats_df.columns)
    if missing_columns:
        raise KeyError(f"The DataFrame is missing the following required columns: {', '.join(missing_columns)}")
    
    # Check if 'gpm' and 'adj_min' are numeric
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['gpm']):
        raise TypeError("'gpm' column must be numeric.")
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['adj_min']):
        raise TypeError("'adj_min' column must be numeric.")
    
    # Calculate 'x_goals'
    lineup_stats_df = lineup_stats_df.copy()
    lineup_stats_df['x_goals'] = lineup_stats_df['gpm'] * lineup_stats_df['adj_min']
    
    return lineup_stats_df

In [27]:
def calculate_league_avg_xg_against_per_60(goalie_stats_df: pd.DataFrame) -> float:
    """
    Calculate the league average expected goals against (xg_against) per 60 minutes.

    Args:
        goalie_stats_df (pd.DataFrame): DataFrame containing goalie statistics with 'xg_against' and 'toi' columns.

    Returns:
        float: The league average xg_against per 60 minutes.

    Raises:
        KeyError: If required columns are missing from the DataFrame.
        ValueError: If no valid goalies with non-zero 'toi' are found.
    """
    # Ensure required columns are present
    required_columns = {'xg_against', 'toi'}
    missing_columns = required_columns - set(goalie_stats_df.columns)
    if missing_columns:
        raise KeyError(f"Missing columns in goalie_stats_df: {', '.join(missing_columns)}")

    # Drop rows with missing or zero 'toi' to avoid division errors
    valid_goalies = goalie_stats_df.dropna(subset=['xg_against', 'toi'])
    valid_goalies = valid_goalies[valid_goalies['toi'] > 0]

    if valid_goalies.empty:
        raise ValueError("No valid goalies with non-zero 'toi' found in goalie_stats_df.")

    # Calculate xg against per 60 minutes for each goalie
    valid_goalies['xg_against_per_60'] = (valid_goalies['xg_against'] / valid_goalies['toi']) * 60

    # Calculate the league average
    league_avg_xg_against_per_60 = valid_goalies['xg_against_per_60'].mean()

    return league_avg_xg_against_per_60

In [28]:
def calculate_xg_against_adj_percentage(lineup_goalie_stats, goalie_avg_xg_against_per_60):
    """
    Calculate the expected goals against adjusted percentage.
    """
    lineup_goalie_stats['adj%'] = (lineup_goalie_stats['xg_against/60'] / goalie_avg_xg_against_per_60) * 100
    return lineup_goalie_stats

In [29]:
def process_team_lineup(input_date: str, team: str, model, poly) -> pd.DataFrame:
    """
    Processes the team's lineup for a given date and predicts GPM for each player.
    This function performs the following steps:
        1. Calls `nst_on_ice_scraper` for the input date minus one day.
        2. Extracts the team's lineup using `extract_team_lineup`.
        3. Retrieves skater statistics with `get_skater_stats`.
        4. Calculates `min%` using `calculate_min_percentage`.
        5. Computes `adj_min` using `calculate_adj_min`.
        6. Determines `ixg_per_60` using `calculate_ixg_per_60`.
        7. Predicts `gpm` using the loaded polynomial regression model.
        8. Calculcates x_goals by multiplying gpm by adj_min    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        team (str): The three-letter team code (e.g., 'TOR').
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        
    Returns:
        pd.DataFrame: Updated DataFrame with calculated metrics and predicted GPM.
    
    Raises:
        ValueError: If any step in the data processing pipeline fails.
    """

    try:
        # Step 1: Calculate the date minus one day
        reference_datetime = datetime.strptime(input_date, '%Y-%m-%d') - timedelta(days=1)
        reference_date_str = reference_datetime.strftime('%Y-%m-%d')
        print(f"Fetching data for reference date: {reference_date_str}")

        # Step 2: Call nst_on_ice_scraper for player and goalie stats
        player_stats_df = nst_on_ice_scraper(
            fromseason=20242025,
            thruseason=20242025,
            startdate='',
            enddate=reference_date_str,
            rate='y',
            stdoi='std'
        )
        goalie_stats_df = nst_on_ice_scraper(
            fromseason=20242025,
            thruseason=20242025,
            startdate='',
            enddate=reference_date_str,
            pos='G',
            rate='y',
            stdoi='g'
        )
        print("Player and goalie statistics fetched successfully.")

        # Step 3: Extract team lineup for the input date and team
        lineup = extract_team_lineup(team, input_date)
        print(f"Lineup extracted for team {team} on {input_date}.")

        # Step 4: Get skater statistics for the lineup
        lineup_skater_stats = get_skater_stats(lineup, player_stats_df)

        # Step 5: Calculate min%
        lineup_skater_stats = calculate_min_percentage(lineup_skater_stats)

        # Step 6: Calculate adj_min
        total_min_percentage = sum_min_percentage(lineup_skater_stats)
        calculate_adj_min(lineup_skater_stats, total_min_percentage)

        # Step 7: Predict gpm using the polynomial regression model
        lineup_skater_stats = add_gpm_to_lineup(lineup_skater_stats, model, poly)

        # Step 8: Calculate x_goals
        lineup_skater_stats = calculate_x_goals(lineup_skater_stats)

        # Step 9: Get goalie stats for the lineup
        lineup_goalie_stats = get_goalie_stats(lineup, goalie_stats_df)
        
        # Step 10: Calculate league average xg_against_per_60
        goalie_avg_xg_against_per_60 = goalie_stats_df['xg_against/60'].mean()

        lineup_goalie_stats = calculate_xg_against_adj_percentage(lineup_goalie_stats, goalie_avg_xg_against_per_60)
        
        return lineup_skater_stats, lineup_goalie_stats

    except Exception as e:
        print(f"An error occurred during processing: {e}")
        raise

In [30]:
def load_models(model_filepath='models/polynomial_model_degree_1.pkl',
               poly_filepath='models/polynomial_features_degree_1.pkl'):
    """
    Loads the regression model and PolynomialFeatures transformer from the specified file paths.

    Args:
        model_filepath (str): Path to the saved regression model pickle file.
        poly_filepath (str): Path to the saved PolynomialFeatures transformer pickle file.

    Returns:
        tuple: A tuple containing the loaded regression model and PolynomialFeatures transformer.

    Raises:
        FileNotFoundError: If either of the specified files does not exist.
        pickle.UnpicklingError: If there is an error unpickling the files.
    """
    try:
        # Load the regression model
        with open(model_filepath, 'rb') as model_file:
            loaded_model = pickle.load(model_file)
        print(f"Model loaded from {model_filepath}")

        # Load the PolynomialFeatures transformer
        with open(poly_filepath, 'rb') as poly_file:
            loaded_poly = pickle.load(poly_file)
        print(f"PolynomialFeatures transformer loaded from {poly_filepath}")

        return loaded_model, loaded_poly

    except FileNotFoundError as fnf_error:
        print(f"Error: {fnf_error}")
        raise
    except pickle.UnpicklingError as pickle_error:
        print(f"Error loading pickle files: {pickle_error}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [31]:
# Load the models using the new function
loaded_model, loaded_poly = load_models()

# Call the function with desired date and team
input_date = '2024-12-02'
team = 'TOR'
lineup_skater_stats, lineup_goalie_stats = process_team_lineup(input_date, team, loaded_model, loaded_poly)


Model loaded from models/polynomial_model_degree_1.pkl
PolynomialFeatures transformer loaded from models/polynomial_features_degree_1.pkl
Fetching data for reference date: 2024-12-01
Player and goalie statistics fetched successfully.
Game ID: 2024020378
Added player 'Mitch Marner' to Forwards slot 1.
Added player 'Steven Lorentz' to Forwards slot 2.
Added player 'Matthew Knies' to Forwards slot 3.
Added player 'Connor Dewar' to Forwards slot 4.
Added player 'Pontus Holmberg' to Forwards slot 5.
Added player 'Auston Matthews' to Forwards slot 6.
Added player 'Fraser Minten' to Forwards slot 7.
Added player 'Nikita Grebenkin' to Forwards slot 8.
Added player 'William Nylander' to Forwards slot 9.
Added player 'Nicholas Robertson' to Forwards slot 10.
Added player 'John Tavares' to Forwards slot 11.
Added player 'Alex Nylander' to Forwards slot 12.
Added player 'Simon Benoit' to Defense slot 1.
Added player 'Chris Tanev' to Defense slot 2.
Added player 'Jake McCabe' to Defense slot 3.
Add

In [32]:
lineup_skater_stats.head()

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%,min%,adj_min,gpm,x_goals
0,Mitch Marner,TOR,R,23,320.07,13.92,0.56,1.69,0.94,0.75,2.25,80.00,6.56,8.57,0.59,12.18,7.69,6.94,1.87,0.56,1.69,0.37,0.19,0.19,0.0,0.0,0.56,3.94,2.06,1.87,1.69,1.69,0.19,0.56,4.69,4.64,18.15,0.009614,0.174491
1,Steven Lorentz,TOR,C,23,208.87,9.08,0.57,0.29,0.00,0.29,0.86,50.00,5.17,11.11,0.81,12.93,9.19,5.46,2.87,0.29,0.86,0.00,0.00,0.00,0.0,0.0,0.86,1.44,1.72,15.80,6.03,5.17,1.44,1.72,13.06,3.03,11.85,0.013170,0.156070
2,Matthew Knies,TOR,L,21,283.08,13.48,1.70,0.42,0.21,0.21,2.12,66.67,6.57,25.81,0.89,11.02,8.90,8.48,5.30,0.42,1.70,1.27,0.64,0.64,0.0,0.0,0.85,1.48,0.64,8.69,5.09,1.27,0.00,0.00,-,4.49,17.57,0.014464,0.254128
3,Connor Dewar,TOR,C,7,68.15,9.74,0.00,0.00,0.00,0.00,0.00,-,5.28,0.00,1.05,14.97,7.92,9.68,5.28,0.00,3.52,0.00,0.00,0.00,0.0,0.0,0.88,4.40,0.88,13.21,7.04,5.28,7.92,15.85,29.35,3.25,12.72,0.017050,0.216881
4,Pontus Holmberg,TOR,R,21,231.12,11.01,0.00,0.78,0.26,0.52,0.78,75.00,3.63,0.00,0.35,9.87,5.97,4.93,2.08,0.26,0.26,2.60,1.30,1.30,0.0,0.0,1.30,2.08,0.78,2.08,7.27,2.34,11.16,10.12,13.61,3.67,14.36,0.005734,0.082338


In [33]:
lineup_goalie_stats

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance,adj%
0,Anthony Stolarz,TOR,13,592.80,45.6,28.54,27.23,0.954,1.32,1.21,2.38,5.67,4.86,0.857,0.81,0.21,7.39,6.88,0.932,0.51,0.37,13.56,13.56,1.000,0.00,0.45,3.44,5.67,39.54,17.77,96.512571
1,Joseph Woll,TOR,8,391.17,48.9,26.23,24.54,0.936,1.69,0.63,1.94,4.76,4.29,0.903,0.46,0.39,6.90,6.60,0.956,0.31,0.51,13.04,12.12,0.929,0.92,-0.49,2.30,4.45,40.03,31.55,78.669911


In [34]:
def process_matchups_for_date( input_date: str, model, poly) -> pd.DataFrame:
    """
    Processes all matchup games for a given date by extracting team lineups and their statistics.
    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
    
    Returns:
        pd.DataFrame: A DataFrame with columns 'date', 'game_id', 'away_team', 'home_team',
                      'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'.
    """
    try:
        # Step 1: Call get_matchup_games with the input date as both arguments
        temp_data = get_matchup_games(input_date, input_date)
        game_ids = temp_data.get('game_ids', {}).get('id', [])
        game_dates = temp_data.get('game_ids', {}).get('date', [])
    
        if not game_ids:
            print(f"No games found for the date {input_date}.")
            return pd.DataFrame(columns=['date', 'game_id', 'away_team', 'home_team',
                                         'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'])
    
        if len(game_ids) != len(game_dates):
            print("Mismatch between number of game IDs and game dates.")
            return pd.DataFrame(columns=['date', 'game_id', 'away_team', 'home_team',
                                         'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'])
    
        results = []
    
        # Step 2: Iterate through each game_id and corresponding game_date
        for game_id, game_date in zip(game_ids, game_dates):
            print(f"\nProcessing Game ID: {game_id} on Date: {game_date}")
    
            # Step 3: Get the cleaned boxscore for the current game
            boxscore = get_game_boxscore(game_id, clean=True)
    
            # Extract away and home team abbreviations
            away_team = boxscore.get('away_team')
            home_team = boxscore.get('home_team')
    
            if not away_team or not home_team:
                print(f"Could not extract teams for Game ID: {game_id}. Skipping.")
                continue
    
            print(f"Away Team: {away_team}, Home Team: {home_team}")
    
            # Step 4: Process lineup for the away team
            print(f"Processing lineup for Away Team: {away_team}")
            away_skater_stats, away_goalie_stats = process_team_lineup(
                game_date,
                away_team,
                model,
                poly
            )
    
            # Step 5: Process lineup for the home team
            print(f"Processing lineup for Home Team: {home_team}")
            home_skater_stats, home_goalie_stats = process_team_lineup(
                game_date,
                home_team,
                model,
                poly
            )
    
            # Step 6: Sum x_goals and extract adj% for away team
            away_x_goals = away_skater_stats['x_goals'].sum()
            away_adj_percentage = away_goalie_stats.iloc[0]['adj%'] if not away_goalie_stats.empty else np.nan
    
            # Step 7: Sum x_goals and extract adj% for home team
            home_x_goals = home_skater_stats['x_goals'].sum()
            home_adj_percentage = home_goalie_stats.iloc[0]['adj%'] if not home_goalie_stats.empty else np.nan
    
            # Step 8: Append the results with the new columns
            results.append({
                'date': game_date,
                'game_id': game_id,
                'away_team': away_team,
                'home_team': home_team,
                'away_x_goals': away_x_goals,
                'away_adj%': away_adj_percentage,
                'home_x_goals': home_x_goals,
                'home_adj%': home_adj_percentage
            })
    
        # Convert results to DataFrame with the specified column order
        results_df = pd.DataFrame(results, columns=[
            'date', 'game_id', 'away_team', 'home_team',
            'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'
        ])
    
        print("\nAll matchups processed successfully.")
        return results_df
    
    except Exception as e:
        print(f"An error occurred while processing matchups for date {input_date}: {e}")
        return pd.DataFrame(columns=[
            'date', 'game_id', 'away_team', 'home_team',
            'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'
        ])

In [35]:
matchups = process_matchups_for_date('2024-12-08', loaded_model, loaded_poly)


Processing Game ID: 2024020436 on Date: 2024-12-08
Away Team: SEA, Home Team: NYR
Processing lineup for Away Team: SEA
Fetching data for reference date: 2024-12-07
Player and goalie statistics fetched successfully.
Game ID: 2024020420
Added player 'Chandler Stephenson' to Forwards slot 1.
Added player 'Matty Beniers' to Forwards slot 2.
Added player 'Tye Kartye' to Forwards slot 3.
Added player 'Brandon Tanev' to Forwards slot 4.
Added player 'Jaden Schwartz' to Forwards slot 5.
Added player 'Jared McCann' to Forwards slot 6.
Added player 'Eeli Tolvanen' to Forwards slot 7.
Added player 'Oliver Bjorkstrand' to Forwards slot 8.
Added player 'Ryan Winterton' to Forwards slot 9.
Added player 'Shane Wright' to Forwards slot 10.
Added player 'Mitchell Stephens' to Forwards slot 11.
Added player 'Andre Burakovsky' to Forwards slot 12.
Added player 'Will Borgen' to Defense slot 1.
Added player 'Adam Larsson' to Defense slot 2.
Added player 'Jamie Oleksiak' to Defense slot 3.
Added player 'Vi

In [36]:
matchups

,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%
0,2024-12-08,2024020436,SEA,NYR,2.079544,87.941677,2.728770,121.424058
1,2024-12-08,2024020437,TBL,VAN,2.347403,116.179830,2.073614,80.277035
2,2024-12-08,2024020438,NYI,OTT,2.232727,90.765492,2.312757,93.992710
3,2024-12-08,2024020439,CBJ,WPG,2.506276,114.969623,2.264688,109.321993
4,2024-12-08,2024020440,COL,NJD,2.920452,96.413123,2.644615,88.345079
5,2024-12-08,2024020441,UTA,PHI,2.523776,123.037667,2.339546,99.236938
6,2024-12-08,2024020442,CGY,DAL,2.566812,100.447145,2.510990,95.202916


In [37]:
def calculate_predicted_goals(matchups_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate predicted goals by multiplying x_goals with opposing team's goalie adjustment percentage.
    
    Args:
        matchups_df: DataFrame containing x_goals and adj% columns from process_matchups_for_date
        
    Returns:
        DataFrame with added predicted goals columns
    """
    # Convert percentages to multipliers (e.g. 105% -> 1.05)
    matchups_df = matchups_df.copy()
    matchups_df['away_adj%'] = matchups_df['away_adj%'] / 100
    matchups_df['home_adj%'] = matchups_df['home_adj%'] / 100
    
    # Calculate predicted goals
    matchups_df['away_pred_goals'] = matchups_df['away_x_goals'] * matchups_df['home_adj%']
    matchups_df['home_pred_goals'] = matchups_df['home_x_goals'] * matchups_df['away_adj%']
    
    return matchups_df

In [38]:
matchups = calculate_predicted_goals(matchups)
matchups

,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%,away_pred_goals,home_pred_goals
0,2024-12-08,2024020436,SEA,NYR,2.079544,0.879417,2.728770,1.214241,2.525066,2.399726
1,2024-12-08,2024020437,TBL,VAN,2.347403,1.161798,2.073614,0.802770,1.884426,2.409121
2,2024-12-08,2024020438,NYI,OTT,2.232727,0.907655,2.312757,0.939927,2.098601,2.099186
3,2024-12-08,2024020439,CBJ,WPG,2.506276,1.149696,2.264688,1.093220,2.739911,2.603704
4,2024-12-08,2024020440,COL,NJD,2.920452,0.964131,2.644615,0.883451,2.580075,2.549756
5,2024-12-08,2024020441,UTA,PHI,2.523776,1.230377,2.339546,0.992369,2.504518,2.878523
6,2024-12-08,2024020442,CGY,DAL,2.566812,1.004471,2.510990,0.952029,2.443680,2.522218


In [39]:
def calculate_win_probabilities(matchups_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate home and away win probabilities based on predicted goals.
    
    Args:
        matchups_df (pd.DataFrame): DataFrame containing 'home_pred_goals' and 'away_pred_goals'.
    
    Returns:
        pd.DataFrame: DataFrame with added 'home_win_prob' and 'away_win_prob' columns, and selected columns.
    """
    HIA = 0.28
    HOME = matchups_df['home_pred_goals']
    AWAY = matchups_df['away_pred_goals']
    
    matchups_df['home_win_prob'] = 1 / (1 + np.exp((-(HIA + HOME - AWAY) / 1.318)))
    matchups_df['away_win_prob'] = 1 - matchups_df['home_win_prob']
    
    return matchups_df[[
        'date', 'game_id', 'away_team', 'home_team', 
        'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%', 
        'away_pred_goals', 'home_pred_goals',
        'away_win_prob', 'home_win_prob'
    ]]

In [40]:
matchups = calculate_win_probabilities(matchups)
matchups

,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%,away_pred_goals,home_pred_goals,away_win_prob,home_win_prob
0,2024-12-08,2024020436,SEA,NYR,2.079544,0.879417,2.728770,1.214241,2.525066,2.399726,0.470698,0.529302
1,2024-12-08,2024020437,TBL,VAN,2.347403,1.161798,2.073614,0.802770,1.884426,2.409121,0.351935,0.648065
2,2024-12-08,2024020438,NYI,OTT,2.232727,0.907655,2.312757,0.939927,2.098601,2.099186,0.446978,0.553022
3,2024-12-08,2024020439,CBJ,WPG,2.506276,1.149696,2.264688,1.093220,2.739911,2.603704,0.472752,0.527248
4,2024-12-08,2024020440,COL,NJD,2.920452,0.964131,2.644615,0.883451,2.580075,2.549756,0.452781,0.547219
5,2024-12-08,2024020441,UTA,PHI,2.523776,1.230377,2.339546,0.992369,2.504518,2.878523,0.378432,0.621568
6,2024-12-08,2024020442,CGY,DAL,2.566812,1.004471,2.510990,0.952029,2.443680,2.522218,0.432408,0.567592
